In [ ]:
import sys
sys.version

In [ ]:
try:
    from google.colab import output
    !pip install rai==0.2.0-dev5
    
except ModuleNotFoundError:
    %load_ext autoreload
    %autoreload 2

In [ ]:
import tqdm
import numpy as np
import skimage.measure
import matplotlib.pyplot as plt

In [ ]:
import pydicom
pydicom.__version__

In [ ]:
import rai
rai.__version__

In [ ]:
import raicontours

from raicontours import TG263

raicontours.__version__

In [ ]:
cfg = raicontours.get_config()

In [ ]:
cfg

In [ ]:
image_paths, structure_path = rai.download_deepmind_example()
structure_path

In [ ]:
grids, image_stack, image_uids = rai.paths_to_image_stack_hfs(cfg=cfg, paths=image_paths)
(z_grid, y_grid, x_grid) = grids

In [ ]:
models = rai.load_models(cfg=cfg)

In [ ]:
predicted_masks = rai.inference(
    cfg=cfg,
    models=models,
    image_stack=image_stack,
    max_batch_size=10,
    step_size=40,
)

In [ ]:
structure_ds = pydicom.read_file(structure_path)
[item.ROIName for item in structure_ds.StructureSetROISequence]

In [ ]:
align_map = {
    "Brainstem": [TG263.Brainstem],
    "Lacrimal-Lt": [TG263.Glnd_Lacrimal_L],
    "Lacrimal-Rt": [TG263.Glnd_Lacrimal_R],
    "Lens-Lt": [TG263.Lens_L],
    "Lens-Rt": [TG263.Lens_R],
    "Optic-Nerve-Lt": [TG263.OpticNrv_L],
    "Optic-Nerve-Rt": [TG263.OpticNrv_R],
    "Orbit-Lt": [TG263.Eye_L],
    "Orbit-Rt": [TG263.Eye_R],
    "Parotid-Lt": [TG263.Parotid_L],
    "Parotid-Rt": [TG263.Parotid_R],
    "Spinal-Cord": [TG263.SpinalCord],
    "Submandibular-Lt": [TG263.Glnd_Submand_L],
    "Submandibular-Rt": [TG263.Glnd_Submand_R],
}
structure_names = list(align_map.keys())

dicom_contours_by_structure = rai.dicom_to_contours_by_structure(
    ds=structure_ds, image_uids=image_uids, structure_names=structure_names
)

In [ ]:
predicted_contours_by_structure = {}

In [ ]:
predicted_contours_by_structure["transverse"] = rai.masks_to_contours_by_structure(
    grids=grids, 
    masks=predicted_masks, 
    structure_names=cfg["structures"], 
    orientation="transverse",
)

aligned_predicted_contours_by_structure = rai.merge_contours_by_structure(
    predicted_contours_by_structure["transverse"], align_map
)

dice = {}
for name in align_map:
    dice[name] = rai.dice_from_contours_by_slice(
        dicom_contours_by_structure[name],
        aligned_predicted_contours_by_structure[name],
    )
    
dice

In [ ]:
vmin = 0.2
vmax = 0.4
figsize = (12, 12)

slice_indices, axis_limits, centre_indices = rai.view_ranges_from_masks(grids, predicted_masks)

In [ ]:
rai.plot_contours_by_structure(
    grids=grids,
    images=image_stack,
    contours_by_structure=predicted_contours_by_structure["transverse"],
    orientation="transverse",
    slice_indices=slice_indices,
    axis_limits=axis_limits,
    structure_names=cfg["structures"],
    figsize=figsize,
    vmin=vmin,
    vmax=vmax,
)

In [ ]:
for orientation in ["coronal", "sagittal"]:
    predicted_contours_by_structure[orientation] = rai.masks_to_contours_by_structure(
        grids=grids, 
        masks=predicted_masks, 
        structure_names=cfg["structures"], 
        orientation=orientation,
    )
    rai.plot_contours_by_structure(
        grids=grids,
        images=image_stack,
        contours_by_structure=predicted_contours_by_structure[orientation],
        orientation=orientation,
        slice_indices=slice_indices,
        axis_limits=axis_limits,
        structure_names=cfg["structures"],
        figsize=figsize,
        vmin=vmin,
        vmax=vmax,
    )